In [2]:
from datetime import datetime
import numpy as np
import pandas as pd
from collections import Counter
import re

In [13]:
first_df = pd.read_json('../data/raw/0 - Scraped Games DF.json')

first_df.shape

(10000, 42)

In [14]:
first_df.to_csv('../data/raw/Scraped for Postgres.csv')

In [15]:
first_df.columns

Index(['app_id', 'title', 'release_date', 'positive_review_percent',
       'number_of_reviews', 'price', 'game_page_link', 'tags', 'date_scraped',
       'developer', 'publisher', 'description', 'interface_languages',
       'full_audio_languages', 'subtitles_languages', 'english', 'schinese',
       'tchinese', 'japanese', 'koreana', 'thai', 'bulgarian', 'czech',
       'danish', 'german', 'spanish', 'latam', 'greek', 'french', 'italian',
       'indonesian', 'hungarian', 'dutch', 'norwegian', 'polish', 'brazilian',
       'romanian', 'russian', 'finnish', 'swedish', 'turkish', 'vietnamese'],
      dtype='object')

In [85]:
series = first_df['developer'].value_counts()
series[0:20]

KOEI TECMO GAMES CO., LTD.    53
Square Enix                   36
CAPCOM Co., Ltd.              30
HeR Interactive               29
Kairosoft Co.,Ltd             28
Milestone S.r.l.              22
Valve                         21
Ubisoft Montreal              20
Humongous Entertainment       19
SEGA                          16
LucasArts                     16
Nippon Ichi Software, Inc.    16
Rebellion                     16
INTI CREATES CO., LTD.        16
Jackbox Games, Inc.           15
Chilla's Art                  15
Ryu Ga Gotoku Studio          14
Nihon Falcom                  14
PopCap Games, Inc.            13
Ubisoft                       13
Name: developer, dtype: int64

In [98]:
pattern = r'liica *'
similar_developers = first_df[first_df['developer'].str.contains(pattern, regex=True)]
similar_developers['developer']

2794    liica inc.
5883    liica inc.
Name: developer, dtype: object

In [79]:
from collections import Counter

In [82]:
types = []

for index, row in first_df.iterrows() :
    types.append(type(row['developer']))

values = Counter(types)
values

Counter({str: 7246, list: 6})

In [92]:
dev_clean_test_df = first_df.copy()

In [93]:
def clean(entry) :
    global results
    disallowed = ['inc', 'Inc', 'inc.', 'Inc.', 'INC', 'ltd', 'Ltd', 'LTD', 'ltd.', 'Ltd.', 'LTD.', 'LLC', 'llc', 'LLC.', 'llc', 'Co.', '']
    cleaned_items = []
    if type(entry) == str :
        entry = re.split(r'[,&]', entry)
    elif entry is None :
        entry = []
    for item in entry :
        cleaned_item = item.strip(' ')
        if cleaned_item not in disallowed :
            cleaned_items.append(cleaned_item)
    results.append(cleaned_items)
    return cleaned_items

In [94]:
results = []
for index, row in dev_clean_test_df.iterrows() :
    fixed_list = clean(row['developer'])
    dev_clean_test_df.at[index, 'developer'] = fixed_list

In [95]:
for item in dev_clean_test_df['developer'] :
    if len(item) > 1 :
        print(item)

['Infinity Ward', 'Raven Software', 'Beenox', 'Treyarch', 'High Moon Studios', 'Sledgehammer Games', 'Activision Shanghai', 'Demonware', 'Toys for Bob']
['EA Canada', 'EA Romania']
['NetherRealm Studios', 'QLOC']
['Treyarch', 'Aspyr (Mac)']
['Firaxis Games', 'Aspyr (Mac)', 'Aspyr (Linux)']
['SgtOkiDoki', 'Vilaskis', 'TheLiquidHorse']
['CREATIVE ASSEMBLY', 'Feral Interactive']
['Vicarious Visions', 'Iron Galaxy Studios']
['iRacing.com Motorsport Simulations', 'iRacing']
['Studio Wildcard', 'Instinct Games', 'Efecto Studios', 'Virtual Basement LLC']
['Nicalis', 'Edmund McMillen']
['QLOC', 'DIMPS']
['Ubisoft Annecy', 'Ubisoft Belgrade', 'Ubisoft Kyiv', 'Ubisoft Pune', 'Ubisoft Odesa']
['Ghost Town Games Ltd.', 'Team17']
['Insomniac Games', 'Nixxes Software']
['343 Industries', 'Splash Damage', 'Ruffian Games', 'Bungie', 'Saber Interactive']
['Ubisoft Montreal', 'Ubisoft Quebec', 'Ubisoft Toronto', 'Blue Byte']
['Gearbox Software', 'Aspyr (Mac)', 'Aspyr (Linux)']
['Paradox Development Stud

In [102]:
for index, row in first_df.iterrows() :
    if "\t" in row['developer'] :
        print(row)

app_id                                                                247350
title                                     Artemis Spaceship Bridge Simulator
release_date                                                    Sep 16, 2013
positive_review_percent                                                 0.88
number_of_reviews                                                      295.0
price                                                                  699.0
game_page_link             https://store.steampowered.com/app/247350/Arte...
tags                                  [599, 492, 19, 1755, 1685, 3859, 3942]
date_scraped                                                      2023-10-10
developer                  \n\t\t\t\t\t\t\t\t\t\t\tSep 16, 2013\t\t\t\t\t...
publisher                                                               None
description                Artemis is designed for anyone who watched Sta...
interface_languages                                                [English]

In [91]:
for item in dev_clean_test_df['developer'] :
    for thing in item :
        if thing == 'Co.':
            print(item)

['Mirari', 'Co.']


In [71]:
disallowed = ['inc', 'Inc', 'inc.', 'Inc.', 'ltd', 'Ltd', 'LTD', 'ltd.', 'Ltd.', 'LTD.']
for index, row in dev_clean_test_df.iterrows() :
    for item in row['developer'] :
        if item in disallowed :
            dev_clean_test_df.at[index, 'developer'] = row['developer'].remove(item)
        

In [73]:
missing_devs = set()
for index, row in first_df.iterrows() :
    if row['developer'] == None :
        missing_devs.add(row['app_id'])

In [76]:
first_df = pd.read_json('../data/interim/cleaning_1012.json', orient='records')

missing_dev_df = first_df[first_df['app_id'].isin(missing_devs)]
missing_dev_df.to_csv('../data/raw/Missing Devs.csv')

In [62]:
first_df.loc[7]

app_id                                                               2195250
title                                                       EA SPORTS FC™ 24
release_date                                                    Sep 28, 2023
positive_review_percent                                                 0.55
number_of_reviews                                                     4899.0
price                                                                 6999.0
game_page_link             https://store.steampowered.com/app/2195250/EA_...
tags                            [701, 1254546, 1775, 7481, 3878, 6730, 5055]
date_scraped                                                      2023-10-03
developer                                             EA Canada & EA Romania
publisher                                                    Electronic Arts
description                EA SPORTS FC™ 24 welcomes you to The World’s G...
interface_languages        [English, French, Italian, German, Spanish - S...

In [43]:
developers_list = []

for item in all_developers :
    developers_list.append(item)

In [44]:
developers_list = sorted(developers_list)

In [ ]:
for item in developers_list :
    print(item)

In [3]:
all_languages = ['schinese', 'tchinese', 'japanese', 'koreana', 'thai', 'bulgarian', 'czech', 'danish', \
                 'german', 'english', 'spanish', 'latam', 'greek', 'french', 'italian', 'indonesian', \
                 'hungarian', 'dutch', 'norwegian', 'polish', 'portugese', 'brazilian', 'romanian', \
                 'russian', 'finnish', 'swedish', 'turkish', 'vietnamese', 'ukranian']

all_counted_non_english_languages = all_languages.copy()
all_counted_non_english_languages.remove('english')
all_counted_non_english_languages.remove('portugese')
all_counted_non_english_languages.remove('ukranian')

In [207]:
index_set = set()

for index, row in first_df.iterrows() :
    for language in all_counted_non_english_languages :
        try:
            if 'a' not in str(row[language]) :
                first_df.loc[index, language] = float(row[language])
        except :
            index_set.add(index)

index_set

In [172]:
float_ids = set()
i = 0
for index, row in first_df.iterrows() :
    while i < 10:
        if type(row['app_id']) != type(first_df.loc[4000, 'app_id']) :
            print(index," ",type(row['app_id']))
            i += 1
            # float_ids.add(index)

len(float_ids)

0   <class 'int'>
0   <class 'int'>
0   <class 'int'>
0   <class 'int'>
0   <class 'int'>
0   <class 'int'>
0   <class 'int'>
0   <class 'int'>
0   <class 'int'>
0   <class 'int'>


0

In [168]:
type(first_df.loc[4000, 'app_id'])

numpy.int64

In [198]:
new_df.to_json('../data/raw/0 - Scraped Games DF.json', orient='records')

In [16]:
langs = ['english', 'schinese',
       'tchinese', 'japanese', 'koreana', 'thai', 'bulgarian', 'czech',
       'danish', 'german', 'spanish', 'latam', 'greek', 'french', 'italian',
       'indonesian', 'hungarian', 'dutch', 'norwegian', 'polish', 'brazilian',
       'romanian', 'russian', 'finnish', 'swedish', 'turkish', 'vietnamese']

In [43]:
touched_app_ids = set()

In [50]:
first_df = test_df.copy()

In [45]:
# FIXING NONE, 'FAILED' VALUES IN LANGUAGE COMMENT COUNTS

test_df = first_df.copy()

for index, row in test_df.iterrows() :
    for lang in langs :
        if row[lang] == 'Failed' :
            test_df.loc[index, lang] = 0
            row[lang] = 0
            touched_app_ids.add(row['app_id'])
        if row[lang] == None :
            test_df.loc[index, lang] = 0
            row[lang] = 0
            touched_app_ids.add(row['app_id'])

len(touched_app_ids)

In [48]:
trouble_df = test_df[test_df['app_id'].isin(troubles)]
trouble_df.to_csv('../data/raw/troubles.csv')

In [18]:
# CREATING LANGS/TYPES FOR EASIER POSTRES INGESTION

for lang in langs :
    print(str(lang)+' bigint,')

english bigint,
schinese bigint,
tchinese bigint,
japanese bigint,
koreana bigint,
thai bigint,
bulgarian bigint,
czech bigint,
danish bigint,
german bigint,
spanish bigint,
latam bigint,
greek bigint,
french bigint,
italian bigint,
indonesian bigint,
hungarian bigint,
dutch bigint,
norwegian bigint,
polish bigint,
brazilian bigint,
romanian bigint,
russian bigint,
finnish bigint,
swedish bigint,
turkish bigint,
vietnamese bigint,


In [54]:
touched_ids = set()

In [55]:
# CLEANUNG UP NEGATIVE COMMENT COUNTS

test_df = first_df.copy()

for index, row in test_df.iterrows() :
    for lang in langs :
        if row[lang] < 0 :
            test_df.loc[index, lang] = 0

for id in touched_ids :
    if id not in neg_ids :
        print(id)

In [56]:
len(touched_ids)

0

In [57]:
neg_df = test_df[test_df['app_id'].isin(neg_ids)]
neg_df.to_csv('../data/raw/neg_ids.csv')

In [58]:
first_df = test_df.copy()

In [60]:
first_df.to_json('../data/interim/cleaning_1012.json', orient='records')

In [103]:
ary = np.empty(10)